In [1]:
import pandas as pd

# Import file .tsv
df = pd.read_csv('breast_msk_2025_clinical_data.tsv', sep='\t')

# Tampilkan nama kolom
print("Nama kolom:")
print(df.columns.tolist())

Nama kolom:
['Study ID', 'Patient ID', 'Sample ID', 'Cancer Type', 'Cancer Type Detailed', 'Ethnicity', 'Fraction Genome Altered', 'Sex', 'Gene Panel', 'Metastatic Site', 'MSI Comment', 'MSI Score', 'MSI Type', 'Mutation Count', 'Oncotree Code', 'Overall Survival (Months)', 'Overall Survival Status', 'Primary Tumor Site', 'Race', 'Sample Class', 'Number of Samples Per Patient', 'Sample coverage', 'Sample Type', 'Somatic Status', 'Tumor Purity']


In [2]:
# Mengubah nilai di kolom 'Overall Survival Status'
df['Overall Survival Status'] = df['Overall Survival Status'].map({'1:DECEASED': 1, '0:LIVING': 0})

# Tampilkan nilai unik setelah mapping untuk verifikasi
print("Nilai unik di kolom 'Overall Survival Status' setelah mapping:")
print(df['Overall Survival Status'].unique())

Nilai unik di kolom 'Overall Survival Status' setelah mapping:
[ 1.  0. nan]


In [6]:
from lifelines import CoxPHFitter
import pandas as pd

# Pilih kolom yang diperlukan
columns_needed = [
    'Overall Survival (Months)',
    'Overall Survival Status',
    'Cancer Type Detailed',
    'Primary Tumor Site',
    'Metastatic Site',
    'MSI Type',
    'Mutation Count',
    'Fraction Genome Altered'
]

# Buat subset data dan drop NaN
data_cox = df[columns_needed].dropna()

# Pastikan tipe data benar
data_cox['Overall Survival (Months)'] = data_cox['Overall Survival (Months)'].astype(float)
data_cox['Overall Survival Status'] = data_cox['Overall Survival Status'].astype(int)

# Encode kolom kategorikal ke dummy variables
categorical_cols = ['Cancer Type Detailed', 'Primary Tumor Site', 'Metastatic Site', 'MSI Type']
data_cox_encoded = pd.get_dummies(data_cox, columns=categorical_cols, drop_first=True)

# Tampilkan info data setelah encoding
print(f"Jumlah sampel setelah drop NaN dan encoding: {len(data_cox_encoded)}")
print("Kolom setelah encoding:")
print(data_cox_encoded.columns.tolist())

# Fit model Cox Proportional Hazards
cph = CoxPHFitter(penalizer=0.1)  # Tambahkan penalizer untuk mengatasi kolinearitas
cph.fit(data_cox_encoded, duration_col='Overall Survival (Months)', event_col='Overall Survival Status')

# Tampilkan summary model
print("\nSummary Model Cox:")
print(cph.summary)

# Uji asumsi proportional hazards
print("\nUji Asumsi Proportional Hazards:")
try:
    results = cph.check_assumptions(data_cox_encoded, p_value_threshold=0.05)
    print(results)
except Exception as e:
    print(f"Error dalam uji asumsi: {e}")

Jumlah sampel setelah drop NaN dan encoding: 1929
Kolom setelah encoding:
['Overall Survival (Months)', 'Overall Survival Status', 'Mutation Count', 'Fraction Genome Altered', 'Cancer Type Detailed_Adenomyoepithelioma of the Breast', 'Cancer Type Detailed_Breast Invasive Cancer, NOS', 'Cancer Type Detailed_Breast Invasive Carcinoma, NOS', 'Cancer Type Detailed_Breast Invasive Carcinosarcoma, NOS', 'Cancer Type Detailed_Breast Invasive Ductal Carcinoma', 'Cancer Type Detailed_Breast Invasive Lobular Carcinoma', 'Cancer Type Detailed_Breast Invasive Mixed Mucinous Carcinoma', 'Cancer Type Detailed_Breast Mixed Ductal and Lobular Carcinoma', 'Cancer Type Detailed_Breast Neoplasm, NOS', 'Cancer Type Detailed_Cancer of Unknown Primary', 'Cancer Type Detailed_Invasive Breast Carcinoma', 'Cancer Type Detailed_Juvenile Secretory Carcinoma of the Breast', 'Cancer Type Detailed_Metaplastic Breast Cancer', 'Cancer Type Detailed_Metaplastic Squamous Cell Carcinoma', 'Primary Tumor Site_Cancer of U

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 1929 total observations, 504 right-censored observations>
         test_name = proportional_hazard_test

---
                                                                      test_statistic    p  -log2(p)
Cancer Type Detailed_Adenomyoepithelioma of the Breast          km              0.56 0.45      1.14
                                                                rank            0.59 0.44      1.17
Cancer Type Detailed_Breast Invasive Cancer, NOS                km              0.00 0.97      0.04
                                                                rank            0.00 0.98      0.03
Cancer Type Detailed_Breast Invasive Carcinoma, NOS             km              1.41 0.24      2.09
                                                                rank            1.43 0.23      2.11
Cancer Type Detailed_Breast Invasive Carcinosarcoma, NOS        km              0.61 0.44      1.20
                                                                rank            0.53 0.47      1.10
Cancer Type Detailed_Breast Invasive Ductal Carcinoma           km              0.16 0.68      0.55
                                                                rank            0.17 0.68      0.56
Cancer Type Detailed_Breast Invasive Lobular Carcinoma          km              0.32 0.57      0.81
                                                                rank            0.33 0.56      0.83
Cancer Type Detailed_Breast Invasive Mixed Mucinous Carcinoma   km              2.10 0.15      2.76
                                                                rank            2.11 0.15      2.77
Cancer Type Detailed_Breast Mixed Ductal and Lobular Carcinoma  km              0.16 0.68      0.55
                                                                rank            0.14 0.71      0.50
Cancer Type Detailed_Breast Neoplasm, NOS                       km              0.01 0.93      0.10
                                                                rank            0.01 0.93      0.10
Cancer Type Detailed_Cancer of Unknown Primary                  km              0.04 0.85      0.24
                                                                rank            0.04 0.84      0.25
Cancer Type Detailed_Invasive Breast Carcinoma                  km              0.24 0.63      0.68
                                                                rank            0.25 0.62      0.69
Cancer Type Detailed_Juvenile Secretory Carcinoma of the Breast km              1.73 0.19      2.40
                                                                rank            1.83 0.18      2.50
Cancer Type Detailed_Metaplastic Breast Cancer                  km              0.69 0.41      1.30
                                                                rank            0.72 0.40      1.33
Cancer Type Detailed_Metaplastic Squamous Cell Carcinoma        km              0.01 0.94      0.09
                                                                rank            0.01 0.93      0.10
Fraction Genome Altered                                         km              0.36 0.55      0.86
                                                                rank            0.26 0.61      0.71
MSI Type_Indeterminate                                          km              0.99 0.32      1.65
                                                                rank            0.99 0.32      1.65
MSI Type_Instable                                               km              1.10 0.29      1.76
                                                                rank            0.92 0.34      1.56
MSI Type_Stable                                                 km              1.50 0.22      2.18
                                                                rank            1.49 0.22      2.17
Metastatic Site_Abdomi



1. Variable 'Metastatic Site_Chest Wall' failed the non-proportional test: p-value is 0.0109.

   Advice: with so few unique values (only 2), you can include `strata=['Metastatic Site_Chest
Wall', ...]` in the call in `.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-varying-covariates
[D]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Modify-the-functional-form
[E]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Stratification

[]
